In [1]:
%pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.1 MB/s eta 0:00:0000:0100:010m
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import nltk
from nltk.corpus import words
import random
from difflib import get_close_matches

API_URL = 'https://wordle.votee.dev:8000/random'

nltk.download('words')  # Download the corpus if needed

[nltk_data] Downloading package words to /Users/Rei/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
def make_guess(guess, seed, size):
    """Get a response from the API for a given guess.
    """
    print(f"Guessing: {guess}")
    response = requests.get(f'{API_URL}?guess={guess}&seed={seed}&size={size}')
    return response.json()

response = make_guess('shelf', 1234, 5)
response

Guessing: shelf


[{'slot': 0, 'guess': 's', 'result': 'absent'},
 {'slot': 1, 'guess': 'h', 'result': 'correct'},
 {'slot': 2, 'guess': 'e', 'result': 'correct'},
 {'slot': 3, 'guess': 'l', 'result': 'absent'},
 {'slot': 4, 'guess': 'f', 'result': 'present'}]

`chars` format:
* `chars[:size]`: correct characters in the right slots, `''` for empty slots.
* `chars[size:]`: present characters with unknown slot.

In [4]:
def chrs(chars, size):
    """Concatenate the characters into a string.
    Replace empty slots with present characters.
    """
    present_chars = chars[size:]
    j = 0
    chars_cpy = chars[:size]
    for i in range(size):
        if j == len(present_chars):
            break
        if not chars_cpy[i]:
            chars_cpy[i] = present_chars[j]
            j += 1
    return "".join(chars_cpy)

chars = ['w', '', 'n', 'n', 'y']
chrs(chars, 4)

'wynn'

In [5]:
def handle_guess(response, chars):
    """Handle a response from the API.
    Return True if the guess is correct.
    """
    size = len(response)
    num_correct = 0

    for item in response:
        c = item["guess"]
        result = item["result"]
        slot = item["slot"]

        if result == "correct":
            # If correct, replace slot with character
            num_correct += 1
            chars[slot] = c
            if c in chars[size:]:
                # If c existed in present characters, remove c
                chars.pop(chars.index(c, size))
        elif result == "present":
            # If present, push to the end of list
            if c not in chars:
                chars.append(c)

    if num_correct == size:
        return True
    return False

chars = [""] * 5
handle_guess(response, chars)
print(chars)

['', 'h', 'e', '', '', 'f']


In [6]:
def suggest_word(input_word, word_list, num_suggestion=10):
    """Get a list of close matches to the input word.
    """
    return get_close_matches(input_word, word_list, n=num_suggestion)

suggest_word("appy", words.words())

['yappy',
 'sappy',
 'pappy',
 'nappy',
 'mappy',
 'happy',
 'happy',
 'gappy',
 'cappy',
 'apply']

In [7]:
def guess_word(seed, size=5, num_init=20, max_round=5, num_suggestion=10):
    """Guess a word using the Wordle API.
    Return the word and the number of requests.

    Args:
        seed (int): The seed for the random number generator.
        size (int): The size of the word.
        num_init (int): The number of initial guesses.
        max_round (int): The maximum number of rounds for guessing.
        num_suggestion (int): The number of suggestions per round.
    """
    num_requests = 0

    # Filter size-letter words
    word_list = [word for word in words.words() if len(word) == size]

    chars = [""] * size

    # Initial guess with random words
    init_words = random.choices(word_list, k=num_init)
    for w in init_words:
        response = make_guess(w, seed, size)
        num_requests += 1
        handle_guess(response, chars)

    for _ in range(max_round):
        keyword = chrs(chars, size)
        suggestions = suggest_word(keyword, word_list, num_suggestion)
        if len(keyword) == size:
            # If all the present chars found,
            # fill the empty slots with present characters
            # and add to suggestions
            suggestions.append(keyword)
        for w in suggestions:
            response = make_guess(w, seed, size)
            num_requests += 1
            if handle_guess(response, chars):
                return w, num_requests

    return None, num_requests

In [11]:
word, num_requests = guess_word(seed=17, size=7)
print(f"Word: {word}")
print(f"Number of requests: {num_requests}")

Guessing: faunule
Guessing: binocle
Guessing: abysmal
Guessing: decidua
Guessing: wakeful
Guessing: disturn
Guessing: stuprum
Guessing: bellyer
Guessing: liparid
Guessing: redwood
Guessing: fiefdom
Guessing: ironish
Guessing: whelked
Guessing: Physcia
Guessing: screeve
Guessing: comamie
Guessing: chrisma
Guessing: silesia
Guessing: haggish
Guessing: chiliad
Guessing: placard
Guessing: packery
Guessing: jackrod
Guessing: epacrid
Guessing: wracker
Guessing: whacker
Guessing: upwards
Guessing: uptrack
Guessing: uptaker
Guessing: uphoard
Guessing: placard
Guessing: tankard
Guessing: spayard
Guessing: pochard
Guessing: palikar
Guessing: packway
Guessing: packman
Guessing: packery
Guessing: package
Guessing: jackrod
Guessing: packard
Word: packard
Number of requests: 41
